# give a seqence, and guess next ABC

In [2]:
import tensorflow as tf
import numpy as np

In [63]:
# build the graph
LEN = 7
n_classes = 3 # guess which value Left will output?

x = tf.placeholder(tf.float32, shape=[None, LEN, 2])
y = tf.placeholder(tf.float32, shape=[None, n_classes])

# layer 1: conv1d
conv1_n = 32 # filter count is 32
conv1 = {"weight": tf.Variable(tf.truncated_normal([3, 2, conv1_n] ,stddev=0.1)),
        "bias": tf.Variable(tf.truncated_normal([1, conv1_n], stddev=0.1))}

conv1_z = tf.nn.conv1d(x, conv1["weight"], stride=1, padding="SAME")
conv1_a = tf.nn.relu(conv1_z + conv1["bias"])
print "conv1 with 1d build done"

# layer 2: fc
# reshape into vector
nodes = conv1_n * LEN
fc1_input = tf.reshape(conv1_a, [-1, nodes])

fc1_n = 32
fc1 = {"weight": tf.Variable(tf.truncated_normal([nodes, fc1_n], stddev=0.1)),
     "bias": tf.Variable(tf.truncated_normal([1, fc1_n]))}
fc1_z = tf.add(tf.matmul(fc1_input, fc1["weight"]), fc1["bias"])
fc1_a = tf.nn.relu(fc1_z)
nodes = fc1_n
print "fc1 layer build done"

# layer 3: softmax
out = {"weight": tf.Variable(tf.truncated_normal([nodes, n_classes], stddev=0.1)),
      "bias": tf.Variable(tf.truncated_normal([1, n_classes], stddev=0.1))}

print "layer 4 build done "
outting = fc1_a
y_ = tf.nn.softmax(tf.add(tf.matmul(outting, out["weight"]), out["bias"]))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))

optm = tf.train.AdamOptimizer().minimize(cost)
corr = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, tf.float32))
# pred = tf.argmax(y_, 1)

print "compute graph ready!"

conv1 with 1d build done
fc1 layer build done
layer 4 build done 
compute graph ready!


In [76]:
# build the rules
def score(l, r):
    return 1 if r > l else -1 if r < l else 0
    
# run a random sequence
def sample(cnt):
    seq = []
    ls = np.random.randint(2, size=cnt)
    rs = np.random.randint(3, size=cnt)
    for i in range(cnt):
        seq.append([ls[i], score(ls[i], rs[i])])
    return seq

## generate the data
def gen(all, since, cnt, window):
    X = []
    Y = []
    for i in range(since, since + cnt):
        x = all[i: i+window, :]
        X.append(x)
        y = [0, 0, 0]
        val = all[i + window][0]
        y[val] = 1
        Y.append(y)
    return np.array(X), np.array(Y)

total = 1000
train_x_count = 700
test_count = 190
seq = np.array(sample(total))

train_X, train_Y = gen(seq, 0, train_x_count, LEN)
test_X, test_Y = gen(seq, train_x_count, test_count, LEN)

print "train x shape", train_X.shape # should be [70, 7, 2]
print "train Y shape", train_Y.shape # should be [70, 3]
print "TEST x shape", test_X.shape # should be [18, 7, 2]
print "TEST y shape", test_Y.shape # should be [18, 3]

assert(np.argmax(train_Y[0, :]) == seq[LEN, 0])
assert(np.argmax(train_Y[1, :]) == seq[LEN + 1, 0])

train x shape (700, 7, 2)
train Y shape (700, 3)
TEST x shape (190, 7, 2)
TEST y shape (190, 3)


In [77]:
# run
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print "init run done"

print "train x shape", train_X.shape
print "train y shape", train_Y.shape
epochs = 1000
print_step = 10
for epoch in range(epochs):
    feeds = {x: train_X, y: train_Y}
    sess.run([optm], feed_dict=feeds)

    # predict in test dataset
    if epoch % print_step == 0:
        feeds = {x: train_X, y: train_Y}
        train_accr = sess.run(accr, feed_dict=feeds)
        feeds = {x: test_X, y: test_Y}
        test_accr = sess.run(accr, feed_dict=feeds)
        print "Epoch %s accuracy (train vs. test)" %epoch, train_accr, test_accr

init run done
train x shape (700, 7, 2)
train y shape (700, 3)
Epoch 0 accuracy (train vs. test) 0.0 0.0
Epoch 10 accuracy (train vs. test) 0.491429 0.5
Epoch 20 accuracy (train vs. test) 0.525714 0.473684
Epoch 30 accuracy (train vs. test) 0.531429 0.484211
Epoch 40 accuracy (train vs. test) 0.515714 0.463158
Epoch 50 accuracy (train vs. test) 0.514286 0.473684
Epoch 60 accuracy (train vs. test) 0.528571 0.447368
Epoch 70 accuracy (train vs. test) 0.554286 0.447368
Epoch 80 accuracy (train vs. test) 0.56 0.426316
Epoch 90 accuracy (train vs. test) 0.575714 0.452632
Epoch 100 accuracy (train vs. test) 0.588571 0.457895
Epoch 110 accuracy (train vs. test) 0.588571 0.468421
Epoch 120 accuracy (train vs. test) 0.585714 0.463158
Epoch 130 accuracy (train vs. test) 0.592857 0.478947
Epoch 140 accuracy (train vs. test) 0.605714 0.494737
Epoch 150 accuracy (train vs. test) 0.62 0.494737
Epoch 160 accuracy (train vs. test) 0.608571 0.505263
Epoch 170 accuracy (train vs. test) 0.625714 0.489474